In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
import re
sensor_folder = Path('.') / 'Original_Data' / 'Sensor Data'
survey_file = Path('.') / 'Original_Data' / 'Survey' / 'Filmtrailer onderzoek juni-1 - MERGED_SURVEY_RESPONSE_MATRIX.txt'
save_to = Path('.') / 'training_data'

In [2]:
def get_survey_data(file_path):
    survey_data = pd.read_csv(file_path.absolute(), sep = "\t", skiprows=57)
    survey_data = survey_data.drop(list(survey_data.filter(regex=("^LABELVALUE|LABELVALUE_Cijfer|TIMING"))), axis=1)
    
    d = {}

    for c in survey_data.columns[5:]:
        splitted = c.split(",")[0]
        trailer_number = re.findall(r"\d+", splitted)[0]
        if "LABELID_Film" in c:
            d[c] = "Seen_Stimulus_" + str(trailer_number)
        elif "LABELID_Cijfer" in c:
            d[c] = "Rating_Stimulus_" + str(trailer_number)
        elif "VALUE Voorspelling" in c:
            d[c] = "Go_watch_Stimulus_" + str(trailer_number)
            
    survey_data = survey_data.rename(columns=d)
    
    return survey_data

survey_data = get_survey_data(survey_file)


display(HTML(survey_data.to_html()))

,STUDY,RESPONDENT,GENDER,AGE,GROUP,Seen_Stimulus_3,Rating_Stimulus_3,Go_watch_Stimulus_3,Seen_Stimulus_7,Rating_Stimulus_7,Go_watch_Stimulus_7,Seen_Stimulus_2,Rating_Stimulus_2,Go_watch_Stimulus_2,Seen_Stimulus_15,Rating_Stimulus_15,Go_watch_Stimulus_15,Seen_Stimulus_9,Rating_Stimulus_9,Go_watch_Stimulus_9,Seen_Stimulus_16,Rating_Stimulus_16,Go_watch_Stimulus_16,Seen_Stimulus_1,Rating_Stimulus_1,Go_watch_Stimulus_1,Seen_Stimulus_6,Rating_Stimulus_6,Go_watch_Stimulus_6,Seen_Stimulus_10,Rating_Stimulus_10,Go_watch_Stimulus_10,Seen_Stimulus_14,Rating_Stimulus_14,Go_watch_Stimulus_14,Seen_Stimulus_12,Rating_Stimulus_12,Go_watch_Stimulus_12,Seen_Stimulus_8,Rating_Stimulus_8,Go_watch_Stimulus_8,Seen_Stimulus_5,Rating_Stimulus_5,Go_watch_Stimulus_5,Seen_Stimulus_4,Rating_Stimulus_4,Go_watch_Stimulus_4,Seen_Stimulus_13,Rating_Stimulus_13,Go_watch_Stimulus_13,Seen_Stimulus_11,Rating_Stimulus_11,Go_watch_Stimulus_11,Unnamed: 133
0,Filmtrailer onderzoek juni,RESPONDENT 1,MALE,40,Default,Ja,9.0,4.0,Nee,5.0,2.0,Ja,7.0,3.0,Nee,1.0,1.0,Ja,5.0,2.0,Ja,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Filmtrailer onderzoek juni,RESPONDENT 6,MALE,46,Default,Nee,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,8.0,1.0,Nee,6.0,2.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,7.0,3.0,NaN,NaN,NaN,Nee,8.0,4.0,NaN
2,Filmtrailer onderzoek juni,RESPONDENT 7,MALE,45,Default,Ja,7.0,4.0,Nee,8.0,5.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,5.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Filmtrailer onderzoek juni,RESPONDENT 10,MALE,42,Default,Nee,7.0,3.0,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,Nee,4.0,2.0,NaN,NaN,NaN,Nee,5.0,2.0,Nee,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Filmtrailer onderzoek juni,RESPONDENT 12,MALE,71,Default,Nee,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,4.0,2.0,Nee,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,4.0,2.0,Nee,6.0,2.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Filmtrailer onderzoek juni,RESPONDENT 15,MALE,70,Default,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,8.0,4.0,Nee,6.0,3.0,Nee,8.0,4.0,NaN,NaN,NaN,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,NaN
6,Filmtrailer onderzoek juni,RESPONDENT 17,MALE,45,Default,Nee,9.0,5.0,Ja,9.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,9.0,4.0,NaN,NaN,NaN,Nee,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,2.0,1.0,Nee,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Filmtrailer onderzoek juni,RESPONDENT 21,FEMALE,44,Default,Nee,9.0,5.0,Nee,9.0,5.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,3.0,NaN,NaN,NaN,Nee,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Filmtrailer onderzoek juni,RESPONDENT 22,FEMALE,32,Default,Nee,8.0,4.0,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,7.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nee,8.0,4.0,Nee,6.0,2.0,NaN,NaN,NaN,Nee,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Filmtrailer onderzoek juni,RESPONDENT 26,FEMALE,20,Default,Nee,6.0,3.0,NaN,NaN,NaN,Ja,9.0,5.0,Nee,6.0,2.0,Nee,7.0,4.0,NaN,NaN,NaN,Nee,7.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,8.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def get_respondent_data(file_path):
    sensor_data = pd.read_csv(file_path, skiprows=5, sep = "\t")
    sensor_data = sensor_data[['StudyName', 'Name', 'Age', 'Gender', 'StimulusName',
       'SlideType', 'Timestamp', 'POz (Raw)', 'Fz (Raw)', 'Cz (Raw)',
       'C3 (Raw)', 'C4 (Raw)', 'F3 (Raw)', 'F4 (Raw)', 'P3 (Raw)', 'P4 (Raw)']]
    
    survey_data = get_survey_data(survey_file)
    
    
    for rowval in sensor_data.StimulusName.unique():
        trailer_number = re.findall(r"\d+", rowval)[0]

        if "STIM" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Stimulus_" + trailer_number, inplace=True)
        elif "Film al gezien" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Question_Seen_Movie_" + trailer_number, inplace=True)

        elif "Cijfer" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Question_Rate_Movie_" + trailer_number, inplace=True)

        elif "Voorspelling" in rowval:
            sensor_data.replace(to_replace={"StimulusName":rowval}, value = "Label_GoSee_Movie_" + trailer_number, inplace=True)
    
    
    participant_survey = survey_data.loc[survey_data.RESPONDENT == sensor_data.Name.unique()[0], :].dropna(axis=1)
    
    participant_labels = participant_survey.loc[:,[col for col in participant_survey if "Rating" in col]]
    
    
    sensor_data = sensor_data.iloc[:,4:]
    unique_stimuli = sensor_data.StimulusName.unique()
    
    
    
    unique_stimuli = [s for s in unique_stimuli if "Stimulus" in s]
    
    sensor_data = sensor_data[sensor_data["StimulusName"].isin(unique_stimuli)]
    
    label_unique = participant_labels[["Rating_" + s for s in unique_stimuli if "Stimulus" in s]]
    label_unique.columns = ['_'.join(s.split('_')[2:]) for s in label_unique.columns] #same format as sensor_data
    label_unique = label_unique.T
    label_unique.columns = ['labels']
    
    
    full_data = {}

    columns_to_extract = sensor_data.columns[3:]
    
    for stim in unique_stimuli:
        
        if "Stimulus" in stim:
            df_ = sensor_data.loc[sensor_data["StimulusName"] == stim, columns_to_extract]#.to_numpy()[0:65000,:]
            full_data[stim] = df_.reset_index(drop=True).copy()
    
    ret1, ret2 = pd.concat(full_data).copy(), label_unique.copy()
    new_index = ['trailer_{}'.format(i) for i in range(1, len(ret1.index.get_level_values(0).unique())+1)]
    ret1.index = ret1.index.set_levels(new_index, level=0)
    ret2.index = new_index
    return ret1, ret2


for i, file_path in enumerate(sensor_folder.iterdir()):
    
    sensor_data, label = get_respondent_data(file_path)
    
    save_file_name = (str(file_path).split('_')[-1].split('.')[0].replace(' ', '')).lower()
    save_file_path = str(save_to / save_file_name)
    print(save_file_path)
    sensor_data.to_csv(save_file_path+'_sensor_data.csv')
    label.to_csv(save_file_path+'_label.csv')

training_data\respondent1
training_data\respondent2
training_data\respondent3
training_data\respondent4
training_data\respondent5
training_data\respondent6
training_data\respondent7
training_data\respondent8
training_data\respondent9
training_data\respondent10
training_data\respondent11
training_data\respondent12
training_data\respondent13
training_data\respondent14
training_data\respondent15
training_data\respondent16
training_data\respondent17
training_data\respondent18
training_data\respondent19
training_data\respondent20
training_data\respondent21
training_data\respondent22
training_data\respondent23
training_data\respondent24
training_data\respondent25
training_data\respondent26
training_data\respondent27
training_data\respondent28
training_data\respondent29
training_data\respondent30
training_data\respondent31
training_data\respondent32
